<a href="https://colab.research.google.com/github/iSanjayYadava/deeplearning-models/blob/master/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
import torch
import numpy as np

In [3]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#from pathlib import Path
#DATA_PATH = Path('/content/gdrive/My\ Drive/data/perceptron_toydata.txt')


In [44]:
data = np.genfromtxt('/content/gdrive/My Drive/data/perceptron_toydata.txt', delimiter='\t')
X , y = data[:,:2] , data[:,2]
y = y.astype(int)
print("X.shape",X.shape)
print('y.shape',y.shape)
print('Label count',np.bincount(y))

X.shape (100, 2)
y.shape (100,)
Label count [50 50]


In [0]:
# Shuffling & train/test split
shuffle_idx = np.arange(y[0])
shuffle_rnd = np.random.RandomState(123)
shuffle_rnd.shuffle(shuffle_idx)

X_train,X_test = X[shuffle_idx[:70]], X[shuffle_idx[70:]]
y_train,y_test = y[shuffle_idx[:70]], X[shuffle_idx[70:]]

In [0]:
class perceptron():
  
  def __init__(self,num_features):
    self.num_features = num_features
    self.weights = torch.zero(num_features,1,dtype=np.float32,device=device)
    self.bias = torch.zero(1,dtype=np.float32,device= device)
    
  def forward(self,x):
    
    